In [1]:
import os
import pandas as pd
import numpy as np
from cycler import cycler
import matplotlib as mpl
import matplotlib.pyplot as plt

FILENAME = 'results/results_.csv'
TESTDIR = 'tests/test_'
PLOTDIR = 'tests/test_/plots'
FORMAT = 'png'

# Create target Directory if don't exist
if not os.path.exists(PLOTDIR):
    os.mkdir(PLOTDIR)
    print("Directory " , PLOTDIR ,  " created ")

In [2]:
''' Merge csv files '''

dlist = []
dirs = [d for d in os.listdir(TESTDIR) if os.path.isdir(os.path.join(TESTDIR,d))]
for d in dirs:
	d = os.path.abspath(TESTDIR+'/'+d)
	dlist += [os.path.join(d,file) for file in os.listdir(d) if file.endswith(".csv")]    
    
df = pd.read_csv(dlist[0], skipinitialspace=True)
for x in range(1,len(dlist)):
    df = df.append(pd.read_csv(dlist[x], skipinitialspace=True))
df.to_csv(FILENAME,index=False)

In [ ]:
filename = 'iters'
df = pd.read_csv(FILENAME, skipinitialspace=True)
dfe = pd.read_csv(TESTDIR+'/'+filename+'_energy.csv', skipinitialspace=True)

In [ ]:
def OptSucc(df):
    ''' No. of optimised structures per method (diff no. of total structures) '''
    ''' Normalised to per cent '''

    ran_succ_list = []
    rat_succ_list = []
    totals = {}

    labels = list(df['method'].unique())
    dicts = {}
    
    # For each method and each case
    for method in labels:
        random = df[df['folder'] == 'random']
        random = random.loc[random['method'] == method]

        rattled = df[df['folder'] == 'rattled']
        rattled = rattled.loc[rattled['method'] == method]

        # Find successful structs
        ran_succ = len([b for b in random['opt_succ'] if b])
        rat_succ = len([b for b in rattled['opt_succ'] if b])

        ran_perc = 0
        if len(random):
            ran_perc = ran_succ*100/len(random)
        rat_perc = 0
        if len(rattled):
            rat_perc = rat_succ*100/len(rattled)

        ran_succ_list.append(ran_perc)
        rat_succ_list.append(rat_perc)

    ''' Plot '''

    y = np.arange(len(labels))  # the label locations
    width = 0.35  # the width of the bars

    fig, ax = plt.subplots(figsize=(15,15))
    ax.grid(zorder=0)
    rects1 = ax.barh(y - width/2, ran_succ_list, width, zorder=3, label='Random') # x% per init category
    rects2 = ax.barh(y + width/2, rat_succ_list, width, zorder=3, label='Rattled')

    # Add some text for labels, title and custom x-axis tick labels, etc.
    for i, v in enumerate(ran_succ_list):
        ax.text(v + 1, i - width/2, str(v), fontsize=15)

    for i, v in enumerate(rat_succ_list):
        ax.text(v + 1, i + width/2, str(v), fontsize=15)

    ax.set_xlim(0,110)
    ax.set_yticks(y)
    ax.tick_params(labelsize=15)
    ax.set_yticklabels(labels)
    ax.invert_yaxis()  # labels read top-to-bottom
    ax.legend(fontsize=15)

    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    fig.suptitle('Successfully Optimised Structures', fontsize=20)

    plt.savefig(PLOTDIR+'/'+'OptSucc', format=FORMAT)
    plt.show()
    
    
def TimeDev(df):
    ''' Variation of Optimisation time '''

    methods = list(df['method'].unique())
    total_succ = df[df['opt_succ'] == True] # times of successful opt/ions
    total_succ = total_succ[['method','opt_time']]

    # successful optimisations per case
    ran_succ = df[df['folder'] == 'random']
    ran_succ = ran_succ.loc[ran_succ['opt_succ'] == True][['method','opt_time']] 
    
    rat_succ = df[df['folder'] == 'rattled']
    rat_succ = rat_succ.loc[rat_succ['opt_succ'] == True][['method','opt_time']]

    # Total
    means = total_succ.groupby(['method']).mean()
    stdevs = total_succ.groupby(['method']).apply(np.std)

    ms = means.join(stdevs, lsuffix='_m', rsuffix='_std')

    # Random
    ran_means = ran_succ.groupby(['method']).mean()
    ran_stdevs = ran_succ.groupby(['method']).apply(np.std)

    ran_ms = ran_means.join(ran_stdevs, lsuffix='_m', rsuffix='_std')
    ms_all = ms.join(ran_ms, rsuffix='_ran')

    # Rattled
    rat_means = rat_succ.groupby(['method']).mean()
    rat_stdevs = rat_succ.groupby(['method']).apply(np.std)

    rat_ms = rat_means.join(rat_stdevs, lsuffix='_m', rsuffix='_std')
    ms_all = ms_all.join(rat_ms, rsuffix='_rat')

    fig, ax = plt.subplots(figsize=(15,15))

    ind = np.arange(len(methods))  # the x locations for the groups
    width = 0.3  # the width of the bars

    ax.grid(zorder=0)
    ax.errorbar(ind - width/3, ms_all['opt_time_m_ran'], ms_all['opt_time_std_ran'], 
                linestyle='None', marker='o', ecolor='red',mfc='red',
                zorder=3, mec='red', label='random')
    ax.errorbar(ind, ms_all['opt_time_m'], ms_all['opt_time_std'], linestyle='None', 
                marker='o', ecolor='blue', mfc='blue',
                zorder=3, mec='blue', label='all')
    ax.errorbar(ind + width/3, ms_all['opt_time_m_rat'], ms_all['opt_time_std_rat'], 
                linestyle='None', marker='o', ecolor='orange', mfc='orange',
                zorder=3, mec='orange', label='rattled')

    ax.set_xticks(ind)
    ax.set_xticklabels([row for row in ms_all.index])
    ax.tick_params(labelsize=15)
    ax.set_ylabel('Time (sec)', fontsize=15)
    ax.legend(fontsize=15)
    start, end = ax.get_ylim()
    ax.set_yticks(np.arange(start, end, 0.5))

    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    fig.suptitle('Optimisation Time Deviation', x=0.524, fontsize=20)

    plt.savefig(PLOTDIR+'/'+'TimeDev', format=FORMAT)
    plt.show()

def MeanEnergyIterI(df,dfe):
    ''' Mean of energy value per iteration '''

    # %pylab
    methods = list(df['method'].unique()) # list of methods
    fig, axs = plt.subplots(3, 1, figsize=(15,15))

    # Random 
    random = df[df['folder'] == 'random']

    xticks = 0
    for method in methods: # for each method 

        ran_m = random.loc[(random['method'] == method)][['structure', # keep random cases
                                                       'opt_succ']].set_index('structure')
        mask = (dfe['method'] == method) & (dfe['folder'] == 'random') # fit method and keep random
                                                                       # from energy iters file
        dfe_m = dfe[mask] # results for this method
        dfe_m = dfe_m.set_index('structure')
        e_res = dfe_m.join(ran_m, on='structure') # join on each struct

        # find successful cases
        e_succ = e_res.loc[e_res['opt_succ'] == True]
        e_succ.drop(['opt_succ'], axis=1)
        mean = e_succ.groupby(['method']).mean().T # mean of each iteration in one column
        mean.columns = ['energy']

        axs[0].plot(mean['energy'], label=method)
        axs[0].legend()
        lmean = len(mean.dropna())
        xticks = lmean if lmean > xticks else xticks

        # find failed cases
        e_fail = e_res.loc[e_res['opt_succ'] == False]
        e_fail.drop(['opt_succ'], axis=1)
        mean = e_fail.groupby(['method']).mean().T # mean of each iteration in one column
        mean.columns = ['energy']

        axs[1].plot(mean['energy'], label=method)
        axs[1].legend()

    axs[0].grid()
    axs[0].set_title('Successful Random')
    axs[0].set_xlim(0, xticks+5)
    axs[0].set_xticks(np.arange(0, xticks+5, 25))
    starty, endy = axs[0].get_ylim()

    axs[1].grid()
    axs[1].set_xlim(0, xticks+5)
    axs[1].set_xticks(np.arange(0, xticks+5, 25))
    axs[1].set_title('Failed Random')


    # Rattled
    rattled = df[df['folder'] == 'rattled']

    xticks = 0
    for method in methods: # for each method keep structures from both dfs

        rat_m = rattled.loc[(rattled['method'] == method)][['structure', # keep rattled cases
                                                       'opt_succ']].set_index('structure')
        mask = (dfe['method'] == method) & (dfe['folder'] == 'rattled') # fit method and keep rattled
                                                                       # from energy iters file
        dfe_m = dfe[mask] # results for this method 
        dfe_m.set_index('structure')
        e_res = dfe_m.join(rat_m, on='structure') # join on each struct

        # find successful cases
        e_succ = e_res.loc[e_res['opt_succ'] == True]
        e_succ.drop(['opt_succ'], axis=1)
        mean = e_succ.groupby(['method']).mean().T # mean of each iteration in one column
        mean.columns = ['energy']

        axs[2].plot(mean['energy'], label=method)
        axs[2].legend()
        lmean = len(mean.dropna())
        xticks = lmean if lmean > xticks else xticks

    axs[2].grid()
    axs[2].set_title('Rattled')
    axs[2].set_xlim(0, xticks+5)
    axs[2].set_xticks(np.arange(0, xticks+5, 5))
    starty, endy = axs[2].get_ylim()

    fig.text(-0.05, 1.7, 'Energy (eV)', horizontalalignment='center',
          verticalalignment='center',rotation=90, transform=plt.gca().transAxes)
    fig.text(0.5, -0.15, 'Steps', horizontalalignment='center',
          verticalalignment='center', transform=plt.gca().transAxes)
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    fig.suptitle('Mean Energy Value per Iteration', x=0.524, fontsize=20)

    plt.savefig(PLOTDIR+'/'+'MeanEnergyIterI', format=FORMAT)
    plt.show()

def MeanLEnergyIterII(df, dfe):
    ''' Mean of energy value per iteration '''
    ''' Log scale '''

    # %pylab

    methods = list(df['method'].unique()) # list of methods
    fig, axs = plt.subplots(3, 1, figsize=(15,15))

    # Random 
    random = df[df['folder'] == 'random']

    xticks = 0
    for method in methods: # for each method keep structures from both dfs

        ran_m = random.loc[(random['method'] == method)][['structure',
                                                       'opt_succ']].set_index('structure')
        mask = (dfe['method'] == method) & (dfe['folder'] == 'random') # fit method and keep random
                                                                           # from energy iters file
        dfe_m = dfe[mask]
        dfe_m = dfe_m.set_index('structure')
        e_res = dfe_m.join(ran_m, on='structure') # join on each struct

        # find successful cases
        e_succ = e_res.loc[e_res['opt_succ'] == True]
        e_succ.drop(['opt_succ'], axis=1)
        mean = e_succ.groupby(['method']).mean().T # mean of each iteration in one column
        mean.columns = ['energy']

        mean = -mean.dropna().apply(abs).apply(np.log10) # apply log to non-NaN

        axs[0].plot(mean['energy'], label=method)
        axs[0].legend()
        lmean = len(mean.dropna())
        xticks = lmean if lmean > xticks else xticks

        # find failed cases
        e_fail = e_res.loc[e_res['opt_succ'] == False]
        e_fail.drop(['opt_succ'], axis=1)
        mean = e_fail.groupby(['method']).mean().T # mean of each iteration in one column
        mean.columns = ['energy']

        mean = -mean.dropna().apply(abs).apply(np.log10) # apply log to non-NaN

        axs[1].plot(mean['energy'], label=method)
        axs[1].legend()

    axs[0].grid()
    axs[0].set_title('Successful Random')
    axs[0].set_xlim(0, xticks+5)
    axs[0].set_xticks(np.arange(0, xticks+5, 25))
    starty, endy = axs[0].get_ylim()

    axs[1].grid()
    axs[1].set_xlim(0, xticks+5)
    axs[1].set_xticks(np.arange(0, xticks+5, 25))
    axs[1].set_title('Failed Random')


    # Rattled
    rattled = df[df['folder'] == 'rattled']

    xticks = 0
    for method in methods: # for each method keep structures from both dfs

        rat_m = rattled.loc[(rattled['method'] == method)][['structure',
                                                       'opt_succ']].set_index('structure')
        mask = (dfe['method'] == method) & (dfe['folder'] == 'rattled') # fit method and keep rattled
                                                                           # from energy iters file
        dfe_m = dfe[dfe['method'] == method] # results for this method 
        dfe_m.set_index('structure')
        e_res = dfe_m.join(rat_m, on='structure') # join on each struct

        # find successful cases
        e_succ = e_res.loc[e_res['opt_succ'] == True]
        e_succ.drop(['opt_succ'], axis=1)
        mean = e_succ.groupby(['method']).mean().T # mean of each iteration in one column
        mean.columns = ['energy']

        mean = -mean.dropna().apply(abs).apply(np.log10) # apply log to non-NaN

        axs[2].plot(mean['energy'], label=method)
        axs[2].legend()
        lmean = len(mean.dropna())
        xticks = lmean if lmean > xticks else xticks

    axs[2].grid()
    axs[2].set_title('Rattled')
    axs[2].set_xlim(0, xticks+5)
    axs[2].set_xticks(np.arange(0, xticks+5, 2))
    starty, endy = axs[2].get_ylim()

    fig.text(-0.05, 1.7, 'Energy (eV)', horizontalalignment='center',
          verticalalignment='center',rotation=90, transform=plt.gca().transAxes)
    fig.text(0.5, -0.15, 'Steps', horizontalalignment='center',
          verticalalignment='center', transform=plt.gca().transAxes)
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    fig.suptitle('Mean Energy Value per Iteration', x=0.524, fontsize=20)

    plt.savefig(PLOTDIR+'/'+'MeanLEnergyIterII', format=FORMAT)
    plt.show()

    
def MeanGnormIterII(df, dfg):
    ''' Mean of gnorm value per iteration '''
    # %pylab

    methods = list(df['method'].unique()) # list of methods
    xticks = 0
    fig, axs = plt.subplots(3, 1, figsize=(15,15))

    # Random 
    random = df[df['folder'] == 'random']

    xticks = 0
    for method in methods: # for each method keep structures from both dfs

        ran_m = random.loc[(random['method'] == method)][['structure',
                                                       'opt_succ']].set_index('structure')
        mask = (dfe['method'] == method) & (dfe['folder'] == 'random') # fit method and keep random
                                                                       # from energy iters file
        dfg_m = dfg[mask] # results for this method 
        dfg_m = dfg_m.set_index('structure')
        e_res = dfg_m.join(ran_m, on='structure') # join on each struct

        # find successful cases
        e_succ = e_res.loc[e_res['opt_succ'] == True]
        e_succ.drop(['opt_succ'], axis=1)
        mean = e_succ.groupby(['method']).mean().T # mean of each iteration in one column
        mean.columns = ['energy']

        axs[0].plot(mean['energy'], label=method)
        axs[0].legend()
        lmean = len(mean.dropna())
        xticks = lmean if lmean > xticks else xticks

        # find failed cases
        e_fail = e_res.loc[e_res['opt_succ'] == False]
        e_fail.drop(['opt_succ'], axis=1)
        mean = e_fail.groupby(['method']).mean().T # mean of each iteration in one column
        mean.columns = ['energy']

        axs[1].plot(mean['energy'], label=method)
        axs[1].legend()

    axs[0].grid()
    axs[0].set_title('Successful Random')
    # axs[0].set_ylim(starty, -300)
    axs[0].set_xlim(0, xticks+5)
    axs[0].set_xticks(np.arange(0, xticks+5, 25))
    starty, endy = axs[0].get_ylim()

    axs[1].grid()
    axs[1].set_xlim(0, xticks+5)
    axs[1].set_xticks(np.arange(0, xticks+5, 25))
    # axs[1].set_ylim(starty, endy)
    axs[1].set_title('Failed Random')
    # axs[1].ticklabel_format(style='plain', axis='y', scilimits=(0,0))


    # Rattled
    rattled = df[df['folder'] == 'rattled']

    xticks = 0
    for method in methods: # for each method keep structures from both dfs

        rat_m = rattled.loc[(rattled['method'] == method)][['structure',
                                                       'opt_succ']].set_index('structure')
        mask = (dfg['method'] == method) & (dfg['folder'] == 'rattled') # fit method and keep rattled
                                                                               # from energy iters file
        dfg_m = dfg[mask] # results for this method 
        dfg_m.set_index('structure')
        e_res = dfg_m.join(rat_m, on='structure') # join on each struct

        # find successful cases
        e_succ = e_res.loc[e_res['opt_succ'] == True]
        e_succ.drop(['opt_succ'], axis=1)
        mean = e_succ.groupby(['method']).mean().T # mean of each iteration in one column
        mean.columns = ['energy']

        axs[2].plot(mean['energy'], label=method)
        axs[2].legend()
        lmean = len(mean.dropna())
        xticks = lmean if lmean > xticks else xticks

    axs[2].grid()
    axs[2].set_title('Rattled')
    axs[2].set_xlim(0, xticks+5)
    axs[2].set_xticks(np.arange(0, xticks+5, 25))
    starty, endy = axs[2].get_ylim()

    fig.text(-0.1, 1.7, 'Gnorm', horizontalalignment='center',
          verticalalignment='center',rotation=90, transform=plt.gca().transAxes)
    fig.text(0.5, -0.15, 'Steps', horizontalalignment='center',
          verticalalignment='center', transform=plt.gca().transAxes)
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    fig.suptitle('Mean Gnorm Value per Iteration', x=0.524, fontsize=20)

    plt.savefig(PLOTDIR+'/'+'MeanGnormIterI', format=FORMAT)
    plt.show()

def MeanGnormIterII(df, dfg):
    ''' Mean of gnorm value per iteration '''
    ''' Log scale '''

    # %pylab

    methods = list(df['method'].unique()) # list of methods
    fig, axs = plt.subplots(3, 1, figsize=(15,15))

    # Random 
    random = df[df['folder'] == 'random']

    xticks = 0
    for method in methods: # for each method keep structures from both dfs

        ran_m = random.loc[(random['method'] == method)][['structure',
                                                       'opt_succ']].set_index('structure')
        mask = (dfg['method'] == method) & (dfg['folder'] == 'random') # fit method and keep random
                                                                       # from energy iters file
        dfg_m = dfg[mask] # results for this method 
        dfg_m = dfg_m.set_index('structure')
        e_res = dfg_m.join(ran_m, on='structure') # join on each struct

        # find successful cases
        e_succ = e_res.loc[e_res['opt_succ'] == True]
        e_succ.drop(['opt_succ'], axis=1)
        mean = e_succ.groupby(['method']).mean().T # mean of each iteration in one column
        mean.columns = ['energy']

        mean = -mean.dropna().apply(abs).apply(np.log10) # apply log to non-NaN

        axs[0].plot(mean['energy'], label=method)
        axs[0].legend()
        lmean = len(mean.dropna())
        xticks = lmean if lmean > xticks else xticks

        # find failed cases
        e_fail = e_res.loc[e_res['opt_succ'] == False]
        e_fail.drop(['opt_succ'], axis=1)
        mean = e_fail.groupby(['method']).mean().T # mean of each iteration in one column
        mean.columns = ['energy']

        mean = -mean.dropna().apply(abs).apply(np.log10) # apply log to non-NaN

        axs[1].plot(mean['energy'], label=method)
        axs[1].legend()

    axs[0].grid()
    axs[0].set_title('Successful Random')
    # axs[0].set_ylim(starty, -300)
    axs[0].set_xlim(0, xticks+5)
    axs[0].set_xticks(np.arange(0, xticks+5, 25))
    starty, endy = axs[0].get_ylim()

    axs[1].grid()
    axs[1].set_xlim(0, xticks+5)
    axs[1].set_xticks(np.arange(0, xticks+5, 25))
    # axs[1].set_ylim(starty, endy)
    axs[1].set_title('Failed Random')
    # axs[1].ticklabel_format(style='plain', axis='y', scilimits=(0,0))


    # Rattled
    rattled = df[df['folder'] == 'rattled']

    xticks = 0
    for method in methods: # for each method keep structures from both dfs

        rat_m = rattled.loc[(rattled['method'] == method)][['structure',
                                                       'opt_succ']].set_index('structure')
        mask = (dfg['method'] == method) & (dfg['folder'] == 'rattled') # fit method and keep rattled
                                                                                   # from energy iters file
        dfg_m = dfg[mask] # results for this method 
        dfg_m.set_index('structure')
        e_res = dfg_m.join(rat_m, on='structure') # join on each struct

        # find successful cases
        e_succ = e_res.loc[e_res['opt_succ'] == True]
        e_succ.drop(['opt_succ'], axis=1)
        mean = e_succ.groupby(['method']).mean().T # mean of each iteration in one column
        mean.columns = ['energy']

        mean = -mean.dropna().apply(abs).apply(np.log10) # apply log to non-NaN

        axs[2].plot(mean['energy'], label=method)
        axs[2].legend()
        lmean = len(mean.dropna())
        xticks = lmean if lmean > xticks else xticks

    axs[2].grid()
    axs[2].set_title('Rattled')
    axs[2].set_xlim(0, xticks+5)
    axs[2].set_xticks(np.arange(0, xticks+5, 25))
    starty, endy = axs[2].get_ylim()

    fig.text(-0.1, 1.7, 'Gnorm', horizontalalignment='center',
          verticalalignment='center',rotation=90, transform=plt.gca().transAxes)
    fig.text(0.5, -0.15, 'Steps', horizontalalignment='center',
          verticalalignment='center', transform=plt.gca().transAxes)
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    fig.suptitle('Mean Gnorm Value per Iteration', x=0.524, fontsize=20)

    plt.savefig(PLOTDIR+'/'+'MeanLGnormIterII', format=FORMAT)
    plt.show()